In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=1c5a9d02f9ced7ae3975354e0f6dc682540765bd813b2079fdf63c0ddb852c3b
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

In [3]:
spark = SparkSession.builder.master('local[*]').appName("Teste Eng").getOrCreate()

In [4]:
spark

## **1.3 Carregamento dos Dados**


In [5]:
dados_resultado = spark.read.csv('resultado.csv', sep=',', header=True, inferSchema=True)

In [6]:
dados_resultado.show()

+-----------+-----------+-----+-------------------+-------------------+
|data_acesso|clientes_id|buyin|               rake|            winning|
+-----------+-----------+-----+-------------------+-------------------+
| 2022-08-14|        505| 25.0|               1.75|              -2.75|
| 2022-08-15|        505| 20.0|                4.0|  8.399999618530273|
| 2022-08-16|        505| 65.0|  5.400000035762787|-19.300000309944153|
| 2022-08-17|        505| 30.0| 2.6500000953674316|              -30.0|
| 2022-08-19|        505|  5.0| 0.3499999940395355|  3.549999952316284|
| 2022-08-20|        505| 15.0|  1.899999976158142|               0.25|
| 2022-08-23|        505| 10.0|  2.950000047683716|-3.0999999046325684|
| 2022-08-30|        505|  5.0|0.30000001192092896|               -5.0|
| 2022-11-20|        505| 43.0|    7.3999999538064|              -33.0|
| 2022-11-21|        505| 15.0|  1.550000011920929| 16.400000393390656|
| 2022-11-23|        505| 69.0| 5.8499999195337296|-29.549999594

In [7]:
dados_resultado.printSchema()

root
 |-- data_acesso: date (nullable = true)
 |-- clientes_id: integer (nullable = true)
 |-- buyin: double (nullable = true)
 |-- rake: double (nullable = true)
 |-- winning: double (nullable = true)



In [8]:
dados_clientes = spark.read.csv('clientes.csv', sep=',', header=True, inferSchema=True)

In [9]:
dados_clientes.show()

+------+----+---------------+-------------------+--------------+-----+
|    id|sexo|data_nascimento|      data_cadastro|        cidade|sigla|
+------+----+---------------+-------------------+--------------+-----+
| 94722|   m|     1982-04-17|2023-03-17 12:41:30|     São Paulo|   SP|
|   505|   m|     1985-03-29|2017-10-11 22:21:33|     São Paulo|   SP|
| 43346|   m|     1991-06-11|2020-12-16 13:08:00|         Belém|   PA|
| 34349|   m|     1991-10-08|2020-03-26 01:47:56|      Capivari|   SP|
|100525|   m|     1992-07-29|2023-03-27 00:19:30|     São Paulo|   SP|
| 97089|   m|     1993-05-24|2023-03-21 13:36:54|     São Paulo|   SP|
| 40554|null|     1984-03-12|2020-07-31 03:30:26|     São Paulo|   SP|
| 84489|   m|     2004-06-17|2023-02-13 22:32:06|     São Paulo|   SP|
| 94679|   m|     1999-04-06|2023-03-17 11:14:28|     São Paulo|   SP|
|  4381|   m|     1996-09-16|2018-05-11 18:01:44|     São Paulo|   SP|
| 99490|   m|     1985-02-04|2023-03-25 15:15:34|     São Paulo|   SP|
| 7413

In [10]:
dados_clientes.printSchema()

root
 |-- id: integer (nullable = true)
 |-- sexo: string (nullable = true)
 |-- data_nascimento: date (nullable = true)
 |-- data_cadastro: timestamp (nullable = true)
 |-- cidade: string (nullable = true)
 |-- sigla: string (nullable = true)



In [11]:
dados_clientes.dropna(how="any").show()

+------+----+---------------+-------------------+--------------+-----+
|    id|sexo|data_nascimento|      data_cadastro|        cidade|sigla|
+------+----+---------------+-------------------+--------------+-----+
| 94722|   m|     1982-04-17|2023-03-17 12:41:30|     São Paulo|   SP|
|   505|   m|     1985-03-29|2017-10-11 22:21:33|     São Paulo|   SP|
| 43346|   m|     1991-06-11|2020-12-16 13:08:00|         Belém|   PA|
| 34349|   m|     1991-10-08|2020-03-26 01:47:56|      Capivari|   SP|
|100525|   m|     1992-07-29|2023-03-27 00:19:30|     São Paulo|   SP|
| 97089|   m|     1993-05-24|2023-03-21 13:36:54|     São Paulo|   SP|
| 84489|   m|     2004-06-17|2023-02-13 22:32:06|     São Paulo|   SP|
| 94679|   m|     1999-04-06|2023-03-17 11:14:28|     São Paulo|   SP|
|  4381|   m|     1996-09-16|2018-05-11 18:01:44|     São Paulo|   SP|
| 99490|   m|     1985-02-04|2023-03-25 15:15:34|     São Paulo|   SP|
| 92872|   m|     1988-11-19|2023-03-14 05:44:39|     São Paulo|   SP|
|11043

In [12]:
dados_resultado.createOrReplaceTempView("resultado")


In [13]:
dados_clientes.createOrReplaceTempView("clientesView")

In [14]:
spark.sql("SELECT * FROM resultado").show()

+-----------+-----------+-----+-------------------+-------------------+
|data_acesso|clientes_id|buyin|               rake|            winning|
+-----------+-----------+-----+-------------------+-------------------+
| 2022-08-14|        505| 25.0|               1.75|              -2.75|
| 2022-08-15|        505| 20.0|                4.0|  8.399999618530273|
| 2022-08-16|        505| 65.0|  5.400000035762787|-19.300000309944153|
| 2022-08-17|        505| 30.0| 2.6500000953674316|              -30.0|
| 2022-08-19|        505|  5.0| 0.3499999940395355|  3.549999952316284|
| 2022-08-20|        505| 15.0|  1.899999976158142|               0.25|
| 2022-08-23|        505| 10.0|  2.950000047683716|-3.0999999046325684|
| 2022-08-30|        505|  5.0|0.30000001192092896|               -5.0|
| 2022-11-20|        505| 43.0|    7.3999999538064|              -33.0|
| 2022-11-21|        505| 15.0|  1.550000011920929| 16.400000393390656|
| 2022-11-23|        505| 69.0| 5.8499999195337296|-29.549999594

In [15]:
spark.sql("SELECT * FROM clientesView").show()

+------+----+---------------+-------------------+--------------+-----+
|    id|sexo|data_nascimento|      data_cadastro|        cidade|sigla|
+------+----+---------------+-------------------+--------------+-----+
| 94722|   m|     1982-04-17|2023-03-17 12:41:30|     São Paulo|   SP|
|   505|   m|     1985-03-29|2017-10-11 22:21:33|     São Paulo|   SP|
| 43346|   m|     1991-06-11|2020-12-16 13:08:00|         Belém|   PA|
| 34349|   m|     1991-10-08|2020-03-26 01:47:56|      Capivari|   SP|
|100525|   m|     1992-07-29|2023-03-27 00:19:30|     São Paulo|   SP|
| 97089|   m|     1993-05-24|2023-03-21 13:36:54|     São Paulo|   SP|
| 40554|null|     1984-03-12|2020-07-31 03:30:26|     São Paulo|   SP|
| 84489|   m|     2004-06-17|2023-02-13 22:32:06|     São Paulo|   SP|
| 94679|   m|     1999-04-06|2023-03-17 11:14:28|     São Paulo|   SP|
|  4381|   m|     1996-09-16|2018-05-11 18:01:44|     São Paulo|   SP|
| 99490|   m|     1985-02-04|2023-03-25 15:15:34|     São Paulo|   SP|
| 7413

In [16]:
freq = spark\
    .sql("""
        SELECT month(data_acesso) AS month, COUNT(rake) AS total_rake
            FROM resultado 
            WHERE month(data_acesso) >= 1
            GROUP BY month
            ORDER BY month
    """)
    
freq\
    .show()


+-----+----------+
|month|total_rake|
+-----+----------+
|    1|      5166|
|    2|      4761|
|    3|      5806|
|    4|      4893|
|    5|      2857|
|    6|      3229|
|    7|      3336|
|    8|      3358|
|    9|      3690|
|   10|      4031|
|   11|      4302|
|   12|      4968|
+-----+----------+



In [22]:
df = spark.sql("select * from resultado re join clientesView cl on(re.clientes_id = cl.id)")
df.show()

+-----------+-----------+-----+-------------------+-------------------+---+----+---------------+-------------------+---------+-----+
|data_acesso|clientes_id|buyin|               rake|            winning| id|sexo|data_nascimento|      data_cadastro|   cidade|sigla|
+-----------+-----------+-----+-------------------+-------------------+---+----+---------------+-------------------+---------+-----+
| 2022-08-14|        505| 25.0|               1.75|              -2.75|505|   m|     1985-03-29|2017-10-11 22:21:33|São Paulo|   SP|
| 2022-08-15|        505| 20.0|                4.0|  8.399999618530273|505|   m|     1985-03-29|2017-10-11 22:21:33|São Paulo|   SP|
| 2022-08-16|        505| 65.0|  5.400000035762787|-19.300000309944153|505|   m|     1985-03-29|2017-10-11 22:21:33|São Paulo|   SP|
| 2022-08-17|        505| 30.0| 2.6500000953674316|              -30.0|505|   m|     1985-03-29|2017-10-11 22:21:33|São Paulo|   SP|
| 2022-08-19|        505|  5.0| 0.3499999940395355|  3.54999995231628

In [18]:
from pyspark.sql.functions import collect_set,sum,avg,max,countDistinct,count

In [19]:
df.groupBy("sexo") \
    .agg(count("winning").alias("count_winning")
     ) \
    .show(truncate=False)

+----+-------------+
|sexo|count_winning|
+----+-------------+
|m   |43704        |
|null|4032         |
|f   |2470         |
+----+-------------+



In [20]:
spark\
    .sql("""
        SELECT *
            FROM resultado
            WHERE winning > 0
    """)\
    .show()

+-----------+-----------+-----+------------------+-------------------+
|data_acesso|clientes_id|buyin|              rake|            winning|
+-----------+-----------+-----+------------------+-------------------+
| 2022-08-15|        505| 20.0|               4.0|  8.399999618530273|
| 2022-08-19|        505|  5.0|0.3499999940395355|  3.549999952316284|
| 2022-08-20|        505| 15.0| 1.899999976158142|               0.25|
| 2022-11-21|        505| 15.0| 1.550000011920929| 16.400000393390656|
| 2022-12-14|        505| 36.0| 5.050000011920929|  32.74999809265137|
| 2022-12-15|        505| 39.0|3.3500000834465027| 100.70000267028809|
| 2022-12-21|        505| 11.0| 1.100000038743019| 12.100000381469727|
| 2022-02-17|      43346| 15.0|0.1599999964237213| 29.489999771118164|
| 2022-03-15|      34349| 38.0| 4.939999938011169|  3.010000228881836|
| 2022-03-17|      34349| 18.0|1.9000000357627869| 22.780000686645508|
| 2022-03-21|      34349|100.0|10.380000114440918|   79.4000015258789|
| 2022